In [8]:
# Last Updated: 26 March 2024, 7.51PM.
# Copyright © 2024 Gedeon Koh All rights reserved.
# No part of this publication may be reproduced, distributed, or transmitted in any form or by any means, including photocopying, recording, or other electronic or mechanical methods, without the prior written permission of the publisher, except in the case of brief quotations embodied in reviews and certain other non-commercial uses permitted by copyright law.
# THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHOR OR COPYRIGHT HOLDER BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
# Use of this program for pranks or any malicious activities is strictly prohibited. Any unauthorized use or dissemination of the results produced by this program is unethical and may result in legal consequences.
# This code have been tested throughly. Please inform the operator or author if there is any mistake or error in the code.
# Any damage, disciplinary actions or death from this material is not the publisher's or owner's fault.
# Run and use this program this AT YOUR OWN RISK.
# Version 0.1

#pip install cv2
#pip install numpy
#pip install cv2

import cv2
import numpy as np
import os

# Function to convert cubic Bezier curve control points to parametric equations
def bezier_to_equations(start, c1, c2, end, img_height):
    """Convert cubic Bezier curve control points to parametric equations to be represented in Desmos."""
    start = (start[0], img_height - start[1])  # Invert y-coordinate to match coordinate system
    c1 = (c1[0], img_height - c1[1])
    c2 = (c2[0], img_height - c2[1])
    end = (end[0], img_height - end[1])

    equation_x = f"((1-t)^3*{start[0]} + 3*(1-t)^2*t*{c1[0]} + 3*(1-t)*t^2*{c2[0]} + t^3*{end[0]})"
    equation_y = f"(1-t)^3*{start[1]} + 3*(1-t)^2*t*{c1[1]} + 3*(1-t)*t^2*{c2[1]} + t^3*{end[1]}"
    return equation_x, equation_y

# Path setup
desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
downloads_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Downloads')
image_filename = 'Hello.png'  # Change this to your image filename
IMAGE_PATH = os.path.join(desktop_path, image_filename)
INACCURACY_VALUE = 0.002  # adjust this to change the accuracy of the approximation. The lower, the more accurate but the more equations. It is recommended not to go over 0.03 (3%), but feel free to experiment.

# Read image
image = cv2.imread(IMAGE_PATH, 1)
img_height = image.shape[0]

# Detect edges
edges = cv2.Canny(image, 100, 200)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Output file path
output_file_path = os.path.join(downloads_path, 'equations.txt')

# Write equations to file
with open(output_file_path, "w") as f:
    for contour in contours:
        epsilon = INACCURACY_VALUE * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        for i in range(len(approx)):
            start = approx[i][0]
            end = approx[(i+1) % len(approx)][0]
            
            dx = end[0] - start[0]
            dy = end[1] - start[1]
            
            if dx == 0: # a vertical line
                f.write(f"x = {start[0]} \\left\\{{{img_height - max(start[1], end[1])} <= y <= {img_height - min(start[1], end[1])}\\right\\}}\n")
            elif dy == 0: # a horizontal line
                slope = dy / dx
                y_intercept = start[1] - slope * start[0]
                f.write(f"y = {slope}*x + {y_intercept} \\left\\{{{min(start[0], end[0])} <= x <= {max(start[0], end[0])}\\right\\}}\n")
            else: # a curve
                c1 = (2*start[0] + end[0]) / 3, (2*start[1] + end[1]) / 3
                c2 = (start[0] + 2*end[0]) / 3, (start[1] + 2*end[1]) / 3
                
                eq_x, eq_y = bezier_to_equations(start, c1, c2, end, img_height)
                f.write(f"({eq_x}, {eq_y})\n")
